In [31]:
import math
import numpy as np

def split(array, nrows, ncols):
    """Split a matrix into sub-matrices."""
    r, h = array.shape
    if r % nrows != 0:
        padding = (math.ceil(r / nrows) * nrows) - r
        array = np.vstack((array, np.zeros((padding, h))))
        r, h = array.shape
    if h % ncols != 0:
        padding = (math.ceil(h / ncols) * ncols) - h
        array = np.hstack((array, np.zeros((r, padding))))
        r, h = array.shape
#     print(array.shape)

    num_x_blocks = math.ceil(r / float(nrows))
    num_y_blocks = math.ceil(h / float(ncols))

    chunks = (array.reshape(h//nrows, nrows, -1, ncols)
                 .swapaxes(1, 2)
                 .reshape(-1, nrows, ncols))
    return np.split(chunks, num_y_blocks)

def gather_blocks_to_pages(splits, num_elem_per_page):
    blocks = np.concatenate(splits)
    pages = []
    i = 0

    while i < len(blocks):
        count = 0
        page = []
        while i < len(blocks) and count + blocks[i].shape[0] * blocks[i].shape[1] <= num_elem_per_page:
            page.append(blocks[i])
            count += blocks[i].shape[0] * blocks[i].shape[1]
            i += 1
        pages.append(page)
        print("Adding {} elems to page {}".format(count, len(pages) - 1))
    return pages

def merge_blocks(blocks, num_blocks_x, num_blocks_y, x, y):
    b_x, b_y = blocks[0].shape
    t_x, t_y = (b_x * num_blocks_x, b_y * num_blocks_y,)
    rows = [np.hstack(blocks[i*num_blocks_y:i*num_blocks_y+num_blocks_y]) for i in range(num_blocks_x)]
    matrix = np.vstack(rows)
    assert matrix.shape[0] == t_x
    r_x = t_x - x
    r_y = t_y - y
    return matrix[:-r_x, :-r_y]

# merge_blocks(x, 2, 2, 4, 4)
# x[0]

def pages_to_blocks(pages):
    blocks = []
    for p in pages:
        blocks.extend(p)
    return blocks

In [35]:
j = np.array([
    [1,2,3,4],
    [5,6,7,8],
    [9,10,11,12],
    [13,14,15,16]
])
m, n = j.shape
# print(j.shape)
x = split(j, 3, 3)
print(x)
bx, by = len(x), x[0].shape[0]
ps = gather_blocks_to_pages(x, 63725) # ~1MB 16 bytes * 63725

x = pages_to_blocks(ps)
merge_blocks(x, bx, by, m, n)

[array([[[ 1.,  2.,  3.],
        [ 5.,  6.,  7.],
        [ 9., 10., 11.]],

       [[ 4.,  0.,  0.],
        [ 8.,  0.,  0.],
        [12.,  0.,  0.]]]), array([[[13., 14., 15.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]],

       [[16.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]]])]
Adding 36 elems to page 0


array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6.,  7.,  8.],
       [ 9., 10., 11., 12.],
       [13., 14., 15., 16.]])